In [1]:
import string
import collections
import re
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [2]:
apple_link = '../../Datasets/SberHack/Task2/Comments/AppStore.csv'
google_link = '../../Datasets/SberHack/Task2/Comments/gp/'

In [3]:
app_store_df = pd.read_csv(apple_link)
cols = app_store_df.columns.to_list()
cols.remove('Review')
cols.remove('Developer Reply')
app_store_df = app_store_df.drop(cols, axis=1)
app_store_df = app_store_df.rename(columns={'Review':'Review Text'})
app_store_df

,Review Text,Developer Reply
0,Не скачивается стала обновлять и зависло прило...,NaN
1,Приложение не открывается после обновления!!!!...,NaN
2,"Добрый день, после последнего обновления пропа...",NaN
3,Не могу скачать так как требует подключения Wi-fi,NaN
4,Почему то не могу оплатить при минусе на балан...,NaN
5,Не могу связаться с поддержкой банка. Пишет чт...,NaN
6,"Сделайте возможность отключать эти истории, он...",NaN
7,А я пожалуй закрою карту и все счета в сбере. ...,NaN
8,"Объясните, почему нужно оплачивать перевод ден...",NaN
9,Дипломированные Мошенники,NaN


In [4]:
google_df = pd.DataFrame({'Review Text':[]})

for d in os.listdir(google_link):
    g_df = pd.read_csv(google_link + d, sep=',')
    cols = g_df.columns.to_list()
    cols.remove('Review Text')
    cols.remove('Developer Reply Text')
    g_df = g_df.drop(cols, axis=1)
    google_df = google_df.append(g_df)

google_df

C:\Users\mdere\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
C:\Users\mdere\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Developer Reply Text,Review Text
0,NaN,NaN
1,NaN,все прекрасно и четко
2,NaN,NaN
3,NaN,Супер)
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,все отлично
8,NaN,NaN
9,NaN,удобный интерфейс


In [5]:
google_df = google_df.dropna()
google_df

,Developer Reply Text,Review Text
135,Отлично! Тогда почему не 5 звезд? :),все ок
204,"Уточните, пожалуйста, в чем именно возникают н...",Не удобен поиск поступлений
213,Когда началась проблема? Стабильно ли воспроиз...,Модераторы когда вы исправите ошибку с входом ...
242,"Если надежно, то можно и 5 звезд поставить :)",надежно
284,"Полная история операций необходима, чтобы вы в...","разработчики ,пожалуйста очень прошу сделайте ..."
446,Обновления необходимы для корректной работы пр...,обновления с каждым разом все тяжелее и тяжелее
605,"Благодарим за отзыв! Рады, что нравится 😎","хорошое приложение, удобное. можно оплачивать ..."
642,"Направьте нам, пожалуйста, скриншот, ваши ФИО,...","Постоянно, ПОСТОЯННО, слетают шаблоны. Сил мои..."
648,В рамках экономного пакета Мобильного банка pu...,подключил пакет экономный. Пуш-уведомления вкл...
667,"Поделитесь своим мнением, какие конкретно изме...","Ужасно неудобно, непонятно, нелогично, куча не..."


In [6]:
app_store_df = app_store_df.rename(columns={'Developer Reply':'Developer Reply Text'})
app_store_df

,Review Text,Developer Reply Text
0,Не скачивается стала обновлять и зависло прило...,NaN
1,Приложение не открывается после обновления!!!!...,NaN
2,"Добрый день, после последнего обновления пропа...",NaN
3,Не могу скачать так как требует подключения Wi-fi,NaN
4,Почему то не могу оплатить при минусе на балан...,NaN
5,Не могу связаться с поддержкой банка. Пишет чт...,NaN
6,"Сделайте возможность отключать эти истории, он...",NaN
7,А я пожалуй закрою карту и все счета в сбере. ...,NaN
8,"Объясните, почему нужно оплачивать перевод ден...",NaN
9,Дипломированные Мошенники,NaN


In [7]:
all_df = app_store_df.append(google_df)
all_df

,Developer Reply Text,Review Text
0,NaN,Не скачивается стала обновлять и зависло прило...
1,NaN,Приложение не открывается после обновления!!!!...
2,NaN,"Добрый день, после последнего обновления пропа..."
3,NaN,Не могу скачать так как требует подключения Wi-fi
4,NaN,Почему то не могу оплатить при минусе на балан...
5,NaN,Не могу связаться с поддержкой банка. Пишет чт...
6,NaN,"Сделайте возможность отключать эти истории, он..."
7,NaN,А я пожалуй закрою карту и все счета в сбере. ...
8,NaN,"Объясните, почему нужно оплачивать перевод ден..."
9,NaN,Дипломированные Мошенники


In [8]:
all_df['Review Text'] = all_df['Review Text'].apply(lambda x: re.sub("[^0-9a-zA-Zа-яА-Я ]", "", x))

In [9]:
all_df

,Developer Reply Text,Review Text
0,NaN,Не скачивается стала обновлять и зависло прило...
1,NaN,Приложение не открывается после обновления Уже...
2,NaN,Добрый день после последнего обновления пропал...
3,NaN,Не могу скачать так как требует подключения Wifi
4,NaN,Почему то не могу оплатить при минусе на балан...
5,NaN,Не могу связаться с поддержкой банка Пишет что...
6,NaN,Сделайте возможность отключать эти истории они...
7,NaN,А я пожалуй закрою карту и все счета в сбере П...
8,NaN,Объясните почему нужно оплачивать перевод дене...
9,NaN,Дипломированные Мошенники


In [10]:
all_df['Review Text'] = all_df['Review Text'].apply(lambda x: np.nan if len(x.split(' ')) <= 5 else x)
all_df = all_df.dropna()
all_df

,Developer Reply Text,Review Text
56,"Рады, что вам понравилось приложение!\nПоясним...",Разработчики молодцы Приложение достойное Но в...
57,У нас никаких сбоев не было зафиксировано. Поз...,Почему я не могу сегодня целый день зайти в пр...
59,"В нашем приложении нет рекламы, есть лишь пред...",Забили рекламой хорошее приложение Отключить н...
60,"Исходя из статистики, большинство наших клиент...",В условиях пандемии ввести комиссию говорит о ...
66,Включить возможность скачивания приложений бол...,Невозможно скачать приложение с новым обновлен...
69,"Уточните, в чем именно заключается проблема? Р...",После обновления ничего не работает то месенд...
73,Вы можете посмотреть push до входа в приложени...,Бесят интернетуведомления и хочется вернуть ст...
74,"Рекомендуем удалить ненужные файлы, если памят...",Требовалось обновление теперь не обновляется и...
76,"В настройках телефона, в разделе ""iTunes Store...",280мб не много для приложенияна айфоне лимит 2...
77,Вероятно в настройках телефона выключена возмо...,Не могу скачать приложение Сбербанк Онлайн Уда...


In [11]:
all_df.to_csv('answer_dataset.csv')

In [24]:
all_df

,Unnamed: 0,Review Text
0,0,Не скачивается стала обновлять и зависло прило...
1,1,Приложение не открывается после обновления Уже...
2,2,Добрый день после последнего обновления пропал...
3,3,Не могу скачать так как требует подключения Wifi
4,4,Почему то не могу оплатить при минусе на балан...
5,5,Не могу связаться с поддержкой банка Пишет что...
6,6,Сделайте возможность отключать эти истории они...
7,7,А я пожалуй закрою карту и все счета в сбере П...
8,8,Объясните почему нужно оплачивать перевод дене...
9,10,Очень плохо стало работать приложение в послед...
